<h2>Visualización y descarga de información de Google Earth Engine</h2>

Para acceder a la API de Google Earth Engine es necesario instalar el ambiente de trabajo (*envs/ee.yml*) con conda, ya que contiene las dependencias necesarias para trabajar con este tipo de objetos. 

Todo inicia con la importación de las librerías *ee* y *geemap*, las cuales corresponden a la API de Earth Engine y de visualización de objetos de ee respectivamente.

In [43]:
import ee
import geemap

La creación del geovisor se realiza con la función Map de geemap, que contiene como argumentos *center* y *zoom*, mientras el primero corresponde a las coordenadas centrales de visualización, el segundo hace parte del nivel de acercamiento del geovisor.

In [44]:
Map = geemap.Map(center=(6.25, -75.6), zoom=11)
Map


Map(center=[6.25, -75.6], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…

Se empiezan a agregar algunos datasets que hacen parte de Earth Engine, entre estos se encuentran un modelo de elevación digital de la USGS-SRTM, una colección de imágenes espectrales de planet con resolución espacial de 4.7 metros. La visualización de estas imágenes presentan como parámetros unos valores de histograma y colores en los objetos llamados _vis.

In [45]:
# Add Earth Engine dataset
dem = ee.Image('USGS/SRTMGL1_003')
nicfi = ee.ImageCollection('projects/planet-nicfi/assets/basemaps/americas')
planet = nicfi.filter(ee.Filter.date('2021-09-01','2021-12-01')).first()
dem_vis = {
    'min': 0,
    'max': 4000,
    'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']
    }
planet_vis = {"bands":["R","G","B"],"min":64,"max":5454,"gamma":1.8}

In [46]:
# Visualizar Earth Engine dataset
Map.add_basemap('Esri National Geographic')
Map.addLayer(dem, dem_vis, 'SRTM DEM', True, 0.5)
Map.addLayer(planet,planet_vis,'2021-09 Planet')

Ahora para visualizar un shapefile que se tenga en la máquina local, se usa la funcionalidad *shp_to_ee* de la librería *geemap*, de este modo se convierte en un objeto tipo *FeautureCollection*, que también es agregado con *addLayer* para el geovisor: *map*. 

In [47]:
# Convertir shapefile a objeto ee y su visualizacion
LaValeria = geemap.shp_to_ee("cuenca/LaValeria_WGS84_25K.shp")
Map.addLayer(LaValeria, {}, 'Cuenca La Valeria')

Finalmente, si hay interés en exportar la imagen intersectada con algún polígono de interés, se realizada convirtiendo el objeto *FeatureCollection* a *geometry*, de forma que sea interpretable por la función *ee_export_image* de la librería *geemap*.

In [48]:
# Exportar capa de planet de ee como un geoTIFF
LaValeriaContour = LaValeria.geometry()
geemap.ee_export_image(planet, "Imagenplanet.tif", 4.7, crs= "EPSG:4326", region=LaValeriaContour)

Generating URL ...
Please wait ...
Data downloaded to /home/caemartinezfo/EarthEngine/data/Imagenplanet.tif
